In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [ ]:
df = pd.read_csv("Train.csv", encoding='utf-8')
df['text'] = df['subject'].fillna('') + " " + df['body'].fillna('')

print("5 dòng đầu của dataset:")
print(df[['id', 'subject', 'body', 'label']].head())


In [ ]:
def preprocess(text):
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip()]
    return ". ".join(lines) + "." if lines else ""


import string

def viet_tokenize(text):
    tokens = word_tokenize(text, format='text').split()
    # loại bỏ token chỉ toàn dấu câu
    tokens = [t for t in tokens if any(c.isalnum() for c in t)]
    return tokens


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)


In [ ]:
with open("vietnamese-stopwords-dash.txt", "r", encoding="utf-8") as f:
    viet_stopwords = [line.strip() for line in f if line.strip()]

vectorizer = TfidfVectorizer(tokenizer=viet_tokenize, stop_words=viet_stopwords)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [ ]:
model = MultinomialNB(alpha=0.5)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))


In [ ]:
from collections import defaultdict


# Hàm đếm số lần xuất hiện
def count_words(texts, labels, target_label):
    counter = defaultdict(int)
    for text, label in zip(texts, labels):
        if label == target_label:
            for w in viet_tokenize(text):
                if w not in viet_stopwords:
                    counter[w] += 1
    return counter


ham_words = count_words(X_train, y_train, 'ham')
spam_words = count_words(X_train, y_train, 'spam')

# Top 10 từ
top_k = 20
ham_top = sorted(ham_words.items(), key=lambda x: x[1], reverse=True)[:top_k]
spam_top = sorted(spam_words.items(), key=lambda x: x[1], reverse=True)[:top_k]

# Vẽ bar chart HAM
plt.figure(figsize=(10, 5))
plt.bar([w for w, _ in ham_top], [c for _, c in ham_top], color='green')
plt.xticks(rotation=45)
plt.title("Top 10 từ xuất hiện nhiều nhất lớp HAM")
plt.ylabel("Số lần xuất hiện")
plt.show()

# Vẽ bar chart SPAM
plt.figure(figsize=(10, 5))
plt.bar([w for w, _ in spam_top], [c for _, c in spam_top], color='red')
plt.xticks(rotation=45)
plt.title("Top 10 từ xuất hiện nhiều nhất lớp SPAM")
plt.ylabel("Số lần xuất hiện")
plt.show()


In [ ]:
subj_input = input("\nNhập Subject (tiêu đề): ")
body_input = input("Nhập Body (nội dung): ")
email_text = subj_input + " " + body_input

X_new = vectorizer.transform([email_text])
pred_label = model.predict(X_new)[0]
pred_prob = np.max(model.predict_proba(X_new))

print("\n--- KẾT QUẢ DỰ ĐOÁN ---")
print(f"Subject:\n{subj_input}")
print(f"Body:\n{body_input}\n")
print(f"Dự đoán: {pred_label}")
print(f"Tỉ lệ (%): {pred_prob * 100:.2f}%")

time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
new_row = pd.DataFrame([{
    "subject": subj_input,
    "body": body_input,
    "pred": pred_label,
    "probability (%)": pred_prob * 100,
    "time": time_now
}])

csv_file = "filter_history.csv"
if os.path.exists(csv_file):
    new_row.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8')
else:
    new_row.to_csv(csv_file, index=False, encoding='utf-8')

print(f"\nĐã lưu dự đoán vào {csv_file}")
print("----------")
print("Lưu ý: Kết quả chỉ mang tính tham khảo. Vui lòng không nhập thông tin nhạy cảm, đường link, hay điền vào form trong email.")

